In [4]:
import os
import scvi
import scanpy as sc
import torch
import scvi
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.pyplot import rc_context
from scipy.stats import median_abs_deviation as mad
torch.set_float32_matmul_precision('medium')

In [9]:
#gse136103_1 = sc.read_h5ad("combined_datasets/GSE136103_combined.h5ad")
#gse136103_1.obs['Isolation'] = 'CD 45-'
#gse136103_2 = sc.read_h5ad("combined_datasets/GSE136103_pos_combined.h5ad")
#gse136103_2.obs['Isolation'] = 'CD 45+'
#adata = sc.concat([gse136103_1,gse136103_2])
#adata.obs_names_make_unique(join='_')
#adata.var_names_make_unique(join='_')
#del gse136103_1
#del gse136103_2
adata = sc.read_h5ad("combined_datasets/GSE225381_combined.h5ad")

In [11]:
ribosomal_genes = pd.read_csv("mouse_ribosomal.csv", header=None)
ribosomal_genes = ribosomal_genes[0]
mt_genes = [gene for gene in adata.var_names if gene.startswith('MT-')]

# Remove ribosomal and mitochondrial genes
ribosomal_genes = list(ribosomal_genes)
mt_genes = list(mt_genes)
genes_to_remove = ribosomal_genes + mt_genes
adata = adata[:, ~adata.var_names.isin(genes_to_remove)].copy()

In [13]:
adata

AnnData object with n_obs × n_vars = 36821 × 17337
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'n_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'outlier', 'mt_outlier', 'doublet', 'Sample', 'Group', 'Diet', 'Time'

In [15]:
adata.layers['counts'] = adata.X.copy()
adata.raw = adata
sc.experimental.pp.highly_variable_genes(
        adata, flavor="pearson_residuals", batch_key = "Sample",n_top_genes=2000
    )

In [17]:
adata = adata[:, adata.var["highly_variable"]].copy()

In [20]:
scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             batch_key = "Sample",
                             categorical_covariate_keys=["Group","Diet"])

In [22]:
model = scvi.model.SCVI(adata)
model.view_anndata_setup()

Anndata setup with scvi-tools version 1.1.6.

Setup via `SCVI.setup_anndata` with arguments:

{
│   'layer': 'counts',
│   'batch_key': 'Sample',
│   'labels_key': None,
│   'size_factor_key': None,
│   'categorical_covariate_keys': ['Group', 'Diet'],
│   'continuous_covariate_keys': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│         n_batch          │   6   │
│         n_cells          │ 36821 │
│ n_extra_categorical_covs │   2   │
│ n_extra_continuous_covs  │   0   │
│         n_labels         │   1   │
│          n_vars          │ 2000  │
└──────────────────────────┴───────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │           adata.layers['counts']           │
│         batch          │          adata.obs['_scvi_batch']          │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
│         labels         │         adata.obs['_scvi_labels']          │
└────────────────────────┴────────────────────────────────────────────┘

                   batch State Registry                   
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location   ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['Sample'] │ GSM7047711 │          0          │
│                     │ GSM7047712 │          1          │
│                     │ GSM7047713 │          2          │
│                     │ GSM7047714 │          3          │
│                     │ GSM7047715 │          4          │
│                     │ GSM7047716 │          5          │
└─────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

           extra_categorical_covs State Registry            
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃  Source Location   ┃  Categories   ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['Diet']  │  western Diet │          0          │
│                    │   chow diet   │          1          │
│                    │               │                     │
│ adata.obs['Group'] │    control    │          0          │
│                    │  post-tumor   │          1          │
│                    │   pre-tumor   │          2          │
│                    │               │                     │
└────────────────────┴───────────────┴─────────────────────┘

In [24]:
model.train()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 217/217: 100%|██████████| 217/217 [12:00<00:00,  3.30s/it, v_num=1, train_loss_step=759, train_loss_epoch=747]

`Trainer.fit` stopped: `max_epochs=217` reached.


Epoch 217/217: 100%|██████████| 217/217 [12:00<00:00,  3.32s/it, v_num=1, train_loss_step=759, train_loss_epoch=747]


In [32]:
adata.obsm['X_scVI'] = model.get_latent_representation()
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)
sc.pp.neighbors(adata, use_rep = 'X_scVI')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution = 1)

/var/folders/3k/phqsvgp15y323p7xbkhjz7300000gn/T/ipykernel_5599/2582969182.py:5: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  sc.tl.leiden(adata, resolution = 1)


In [34]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)

In [39]:
import celltypist
predictions = celltypist.annotate(adata, model = 'Healthy_Mouse_Liver.pkl', majority_voting = True)

🔎 No available models. Downloading...
📜 Retrieving model list from server https://celltypist.cog.sanger.ac.uk/models/models.json
📚 Total models in list: 52
📂 Storing models in /Users/tdarde/.celltypist/data/models
💾 Downloading model [1/52]: Immune_All_Low.pkl
💾 Downloading model [2/52]: Immune_All_High.pkl
💾 Downloading model [3/52]: Adult_COVID19_PBMC.pkl
💾 Downloading model [4/52]: Adult_CynomolgusMacaque_Hippocampus.pkl
💾 Downloading model [5/52]: Adult_Human_MTG.pkl
💾 Downloading model [6/52]: Adult_Human_PancreaticIslet.pkl
💾 Downloading model [7/52]: Adult_Human_PrefrontalCortex.pkl
💾 Downloading model [8/52]: Adult_Human_Skin.pkl
💾 Downloading model [9/52]: Adult_Mouse_Gut.pkl
💾 Downloading model [10/52]: Adult_Mouse_OlfactoryBulb.pkl
💾 Downloading model [11/52]: Adult_Pig_Hippocampus.pkl
💾 Downloading model [12/52]: Adult_RhesusMacaque_Hippocampus.pkl
💾 Downloading model [13/52]: Autopsy_COVID19_Lung.pkl
💾 Downloading model [14/52]: COVID19_HumanChallenge_Blood.pkl
💾 Downloadi

In [41]:
adata = predictions.to_adata()

In [43]:
adata.X = adata.layers["scvi_normalized"]

In [45]:
adata.obs

,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,n_counts,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,...,Group,Diet,Time,_scvi_batch,_scvi_labels,leiden,predicted_labels,over_clustering,majority_voting,conf_score
AAACCCAGTCCGTTTC-1,1920,7.560601,4218.0,8.347353,17.255274,4219.0,31.0,3.465736,0.734945,26.0,...,control,chow diet,40 weeks,0,0,0,Hepatocytes,198,Hepatocytes,0.825969
AAACCCAGTGAAAGTT-1,472,6.159095,649.0,6.476973,18.798151,649.0,9.0,2.302585,1.386749,8.0,...,control,chow diet,40 weeks,0,0,1,Hepatocytes,142,Hepatocytes,0.039303
AAACCCAGTTGCGTAT-1,1505,7.317212,2513.0,7.829630,15.791567,2514.0,9.0,2.302585,0.358138,9.0,...,control,chow diet,40 weeks,0,0,4,Fibroblasts,6,Fibroblasts,1.000000
AAACCCATCGTCGACG-1,3548,8.174421,10445.0,9.253974,14.992820,10445.0,47.0,3.871201,0.449976,36.0,...,control,chow diet,40 weeks,0,0,10,Hepatocytes,28,Hepatocytes,0.995105
AAACGAAAGAGCATCG-1,588,6.378426,784.0,6.665684,15.051020,784.0,5.0,1.791759,0.637755,4.0,...,control,chow diet,40 weeks,0,0,0,Hepatocytes,214,Hepatocytes,0.182756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGGCTCCCA-1,495,6.206576,997.0,6.905753,44.433300,997.0,10.0,2.397895,1.003009,20.0,...,post-tumor,western Diet,48 weeks,5,0,1,Kupffer cells,36,Kupffer cells,0.012588
TTTGTTGAGGTTCCAT-1,4236,8.351611,14285.0,9.567036,14.467698,14287.0,202.0,5.313206,1.414071,77.0,...,post-tumor,western Diet,48 weeks,5,0,6,Hepatocytes,32,Hepatocytes,0.998053
TTTGTTGAGGTTCTTG-1,4376,8.384119,14700.0,9.595671,13.136054,14700.0,44.0,3.806663,0.299320,60.0,...,post-tumor,western Diet,48 weeks,5,0,6,Hepatocytes,38,Hepatocytes,0.999204
TTTGTTGCAGTACTAC-1,584,6.371612,1153.0,7.050990,45.099740,1153.0,5.0,1.791759,0.433651,21.0,...,post-tumor,western Diet,48 weeks,5,0,16,B cells,118,B cells,0.058574


In [47]:
adata.obs["CellType"] = adata.obs["majority_voting"]

In [49]:
adata = adata.raw.to_adata()

In [51]:
adata.layers["counts"] = adata.X.copy()

In [53]:
del adata.uns['log1p']

In [55]:
sc.pp.normalize_total(adata, target_sum = 1e4)

In [57]:
sc.pp.log1p(adata)

In [59]:
sc.tl.rank_genes_groups(adata, 'CellType')

In [60]:
adata.layers["scvi_normalized"] = adata.X

In [63]:
adata.write_h5ad("GSE225381"+'.h5ad')